In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import re
import io
from config import api_key_prism
from collections import OrderedDict
from pandas.io.json import json_normalize  
import time 

### Record Requests

In [10]:
## API request from Prism

url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records'

# records = 100
# size = 100 - total
# 'size': 100, 
# 'offset': 100,
#         'page': 2, 
#         'status': 
#         'published', 
#         'sort': 'bestmatch', 
params={

        'all_versions': 'false', 
        'access_token': api_key_prism
       }

response = requests.get(url, params=params)


print(response.url)
print(response.status_code)

response_json = response.json()['hits']
print(json.dumps(response_json, indent=4, sort_keys=True))

https://vtfsmghslrepo02.fsm.northwestern.edu/api/records?all_versions=false&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
{
    "hits": [
        {
            "access": {
                "embargo": {
                    "active": false,
                    "reason": null
                },
                "files": "public",
                "record": "public",
                "status": "open"
            },
            "created": "2022-07-22T17:32:44.207472+00:00",
            "files": {
                "enabled": true,
                "order": []
            },
            "id": "vf4zr-thb15",
            "is_draft": false,
            "is_published": true,
            "links": {
                "access_links": "https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15/access/links",
                "doi": "https://doi.org/10.24418/vf4zr-thb15",
                "draft": "https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15/dra

In [13]:
total = response.json()['hits']['total']
print(total)
len(response_json['hits'])

5601


25

In [ ]:
# import requests

# ## Starting URL
# url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records'
# offset = 0
# size = 100
# records=[]


# while True: 
    
#     ## Python time method sleep() suspends execution for the given number of seconds 
#     time.sleep(0.1) 
    
#     print("------")
    
#     params={'size': size, 
#         'offset': offset,
# #         'page': 2, 
#         'status': 
#         'published', 
#         'sort': 'bestmatch', 
#         'all_versions': 'false', 
#         'access_token': api_key_prism
#        }

#     response = requests.get(url, params=params)
    
#     print(response.url)
#     print(response.status_code)
    
#     response_json = response.json()['hits']
# #     total = response.json()['hits']['total']

    
#     ## Did we find any records? 
#     if len(response_json["hits"]) == 0:
#         ## If no, then break and break the while loop.
#         break
    
#     ## If we did find records, add them to the list and move on to next offset
#     records.extend(response_json["hits"])
#     offset= offset + 100
   
# ## Pros: very flexible
# ## Cons: extra request, doesn't really listen to the API for the next URL

In [14]:
records = response.json()['hits']
print(records)

{'hits': [{'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/records/vf4zr-thb15', 'self_doi': 'https://vtfsmghslrepo02.fsm.northwestern.edu/doi/10.24418/vf4zr-thb15', 'doi': 'https://doi.org/10.24418/vf4zr-thb15', 'self_iiif_manifest': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/iiif/record:vf4zr-thb15/manifest', 'self_iiif_sequence': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/iiif/record:vf4zr-thb15/sequence/default', 'files': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15/files', 'latest': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15/versions/latest', 'latest_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/records/vf4zr-thb15/latest', 'draft': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15/draft', 'versions': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records/vf4zr-thb15/versions', 'access_l

In [16]:
print(len(records))
print(type(records))

2
<class 'dict'>


In [17]:
# prism_item_df = pd.DataFrame.from_dict(json_normalize(response_json, meta=['hits']))
# prism_item_df.head()

prism_item_df = pd.json_normalize(records, max_level = 1)
prism_item_df.head()

,hits,total
0,[{'links': {'self': 'https://vtfsmghslrepo02.f...,5601


In [18]:
remove_hits_nest = prism_item_df.apply(lambda x: pd.Series(x['hits']),axis=1).stack().reset_index(level=1, drop=True).to_frame()
remove_hits_nest.rename(columns = {0:'data'}, inplace = True)
remove_hits_nest.head()


,data
0,{'links': {'self': 'https://vtfsmghslrepo02.fs...
0,{'links': {'self': 'https://vtfsmghslrepo02.fs...
0,{'links': {'self': 'https://vtfsmghslrepo02.fs...
0,{'links': {'self': 'https://vtfsmghslrepo02.fs...
0,{'links': {'self': 'https://vtfsmghslrepo02.fs...


In [ ]:
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

## https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

In [19]:
final_df = pd.DataFrame([flatten_json(x) for x in remove_hits_nest['data']])
final_df.head()

,links_self,links_self_html,links_self_doi,links_doi,links_self_iiif_manifest,links_self_iiif_sequence,links_files,links_latest,links_latest_html,links_draft,...,metadata_dates_0_description,metadata_additional_descriptions_0_type_id,metadata_additional_descriptions_0_type_title_en,metadata_additional_descriptions_0_description,metadata_locations_features_0_place,metadata_locations_features_1_place,metadata_description,metadata_subjects_2_id,metadata_subjects_11_subject,metadata_subjects_11_id
0,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/d...,https://doi.org/10.24418/vf4zr-thb15,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/d...,https://doi.org/10.24418/g3-b92h-jd51,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/d...,https://doi.org/10.15844/pedneurbriefs-24-11-7,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,When the item was originally created.,abstract,Abstract,The effects of attention-deficit/hyperactivity...,NaN,NaN,NaN,NaN,NaN,NaN
3,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/d...,https://doi.org/10.15844/pedneurbriefs-22-12-11,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,When the item was originally created.,abstract,Abstract,Twenty-four adolescents with familial ADHD and...,NaN,NaN,NaN,NaN,NaN,NaN
4,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/d...,https://doi.org/10.15844/pedneurbriefs-18-10-5,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/r...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,When the item was originally created.,abstract,Abstract,The ethical issues raised by families involved...,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
len(final_df)

25

In [ ]:
## Export file to excel, without the Pandas index, but with the header

final_df.to_excel("outputs/final_df.xlsx", index=False, header=True)

In [ ]:
# ## Remove NESTS manually

# remove_0_nest = pd.concat([remove_hits_nest.drop(['data'], axis=1), remove_hits_nest['data'].apply(pd.Series)], axis=1, join="outer")

# ## REMOVE PIDS NESTS
# remove_pids_nest_1 = pd.concat([remove_0_nest.drop(['pids'], axis=1), remove_0_nest['pids'].apply(pd.Series)], axis=1, join="outer")
# remove_pids_nest_1.rename(columns = {'oai':'pids.oai'}, inplace = True)

# remove_pids_nest_2 = pd.concat([remove_pids_nest_1.drop(['pids.oai'], axis=1), remove_pids_nest_1['pids.oai'].apply(pd.Series)], axis=1, join="outer")
# remove_pids_nest_2.rename(columns = {'identifier':'pids.oai.identifer', 'provider': 'pids.oai.provider'}, inplace = True)


# ## REMOVE PARENT NESTS
# remove_parent_nest_1 = pd.concat([remove_pids_nest_2.drop(['parent'], axis=1), remove_pids_nest_2['parent'].apply(pd.Series)], axis=1, join="outer")
# remove_parent_nest_1.rename(columns = {'id':'parent.id', 'communities':'parent.communities'}, inplace = True)

# remove_parent_nest_2 = pd.concat([remove_parent_nest_1.drop(['parent.communities'], axis=1), remove_parent_nest_1['parent.communities'].apply(pd.Series)], axis=1, join="outer")
# remove_parent_nest_2.rename(columns = {'ids':'parent.communities.ids', 'default':'parent.communities.default'}, inplace = True)

# remove_access_nest_1 = pd.concat([remove_parent_nest_1.drop(['access'], axis=1), remove_parent_nest_1['access'].apply(pd.Series)], axis=1, join="outer")
# remove_access_nest_1.rename(columns = {'status':'access.status', 'files':'access.files', 'embargo':'access.embargo', 'record':'access.record'}, inplace = True)


# remove_parent_nest_2.head()

### Communitiy Requests

In [51]:
## API request from Prism for public communities

url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities'

params={
        'accept': 'application/json',
        'access_token': api_key_prism
       }

community_response = requests.get(url, params=params)
print(community_response.url)
print(community_response.status_code)

## Use the .json() to return a JSON object of the result (if the result was written in JSON format)
## The result is a python dictionary
## Resource: https://www.geeksforgeeks.org/response-methods-python-requests/
communtiy_response_json = community_response.json()

## Next, pass the Python dictionary to json.dumps() function, which will return a string. 
print(json.dumps(communtiy_response_json, indent=4, sort_keys=True))

https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities?accept=application%2Fjson&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
{
    "aggregations": {
        "type": {
            "buckets": [
                {
                    "doc_count": 13,
                    "is_selected": false,
                    "key": "project",
                    "label": "Project"
                },
                {
                    "doc_count": 11,
                    "is_selected": false,
                    "key": "organization",
                    "label": "Organization"
                },
                {
                    "doc_count": 9,
                    "is_selected": false,
                    "key": "topic",
                    "label": "Topic"
                },
                {
                    "doc_count": 4,
                    "is_selected": false,
                    "key": "event",
                    "label": "Event"
               

In [53]:
## Using the python dictionary, go two levels deep (i.e. Hits, then  hits) to grab the community record data
community_records = communtiy_response_json['hits']['hits']
print(community_records)

[{'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/covid-19-community', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/covid-19-community/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/members', 'public_members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/members/public', 'invitations': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/invitations', 'requests': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/commun

In [54]:
## Use pandas json_normalize() to normalize semi-structured JSON data into a flat table.
## Max number of levels(depth of dict) to normalize. if None, normalizes all levels.
## Resource: https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
prism_community_nested_df = pd.json_normalize(community_records)
prism_community_nested_df.head()

,created,slug,updated,revision_id,id,links.self,links.self_html,links.settings_html,links.logo,links.rename,...,links.invitations,links.requests,metadata.type.id,metadata.type.title.en,metadata.description,metadata.title,access.record_policy,access.visibility,access.member_policy,metadata.website
0,2022-07-19T14:57:42.661941+00:00,covid-19-community,2022-07-19T14:57:42.721375+00:00,2,9c2f15c3-fe86-492e-957b-ede579f23e84,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,topic,Topic,The COVID-19 Community houses resources on the...,COVID-19 Community,open,public,open,NaN
1,2022-07-19T14:57:42.567187+00:00,food-allergy-data-dictionary,2022-07-19T14:57:42.614680+00:00,2,f881b5bc-5f27-434a-8ad9-a2d5f06964a9,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,FA research lacks a widely adopted terminologi...,The Food Allergy Data Dictionary,open,public,open,NaN
2,2022-07-19T14:57:42.428426+00:00,syncropatch-db-r-package,2022-07-19T14:57:42.501299+00:00,2,e780b050-d348-4e11-8489-0f464cac74c4,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,NaN,Syncropatch DB R Package,open,public,open,NaN
3,2022-07-19T14:57:42.177728+00:00,script-project,2022-07-19T14:57:42.229279+00:00,2,507f92f1-6152-47ec-b8d8-b065d05e71e6,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,These are various artifacts from the SCRIPT pr...,SCRIPT Project,open,public,open,NaN
4,2022-07-19T14:57:42.068743+00:00,grupo-de-equilibrio-de-estilo-de-vida,2022-07-19T14:57:42.131976+00:00,2,84fc9255-0d2c-48ab-aaf2-83a75ae4a113,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,topic,Topic,NaN,Grupo de Equilibrio De Estilo De Vida,open,public,open,NaN


In [59]:
## Export file to excel, without the Pandas index, but with the header

prism_community_nested_df.to_excel("outputs/public_communities.xlsx", index=False, header=True)

In [55]:
## API request from Prism for restricted communities

url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities'

params={
        'accept': 'application/json',
        "access": {
            "visibility": 'restricted'
            },
        'access_token': api_key_prism
       }

restricted_community_response = requests.get(url, params=params)
print(restricted_community_response.url)
print(restricted_community_response.status_code)

## Use the .json() to return a JSON object of the result (if the result was written in JSON format)
## The result is a python dictionary
## Resource: https://www.geeksforgeeks.org/response-methods-python-requests/
restricted_communtiy_response_json = restricted_community_response.json()

## Next, pass the Python dictionary to json.dumps() function, which will return a string. 
print(json.dumps(restricted_communtiy_response_json, indent=4, sort_keys=True))

https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities?accept=application%2Fjson&access=visibility&access_token=mgS5In0TwRjbTbfr1Z72Pbj8bgAFVE4kMrOMNkw0LOLJgwzJK9sHcoOwColI
200
{
    "aggregations": {
        "type": {
            "buckets": [
                {
                    "doc_count": 13,
                    "is_selected": false,
                    "key": "project",
                    "label": "Project"
                },
                {
                    "doc_count": 11,
                    "is_selected": false,
                    "key": "organization",
                    "label": "Organization"
                },
                {
                    "doc_count": 9,
                    "is_selected": false,
                    "key": "topic",
                    "label": "Topic"
                },
                {
                    "doc_count": 4,
                    "is_selected": false,
                    "key": "event",
                    "label": "Even

In [56]:
## Using the python dictionary, go two levels deep (i.e. Hits, then  hits) to grab the community record data
restricted_community_records = restricted_communtiy_response_json['hits']['hits']
print(restricted_community_records)

[{'links': {'self': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84', 'self_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/covid-19-community', 'settings_html': 'https://vtfsmghslrepo02.fsm.northwestern.edu/communities/covid-19-community/settings', 'logo': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/logo', 'rename': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/rename', 'members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/members', 'public_members': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/members/public', 'invitations': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/communities/9c2f15c3-fe86-492e-957b-ede579f23e84/invitations', 'requests': 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/commun

In [57]:
## Use pandas json_normalize() to normalize semi-structured JSON data into a flat table.
## Max number of levels(depth of dict) to normalize. if None, normalizes all levels.
## Resource: https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
prism_restricted_community_nested_df = pd.json_normalize(restricted_community_records)
prism_restricted_community_nested_df.head()

,created,slug,updated,revision_id,id,links.self,links.self_html,links.settings_html,links.logo,links.rename,...,links.invitations,links.requests,metadata.type.id,metadata.type.title.en,metadata.description,metadata.title,access.record_policy,access.visibility,access.member_policy,metadata.website
0,2022-07-19T14:57:42.661941+00:00,covid-19-community,2022-07-19T14:57:42.721375+00:00,2,9c2f15c3-fe86-492e-957b-ede579f23e84,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,topic,Topic,The COVID-19 Community houses resources on the...,COVID-19 Community,open,public,open,NaN
1,2022-07-19T14:57:42.567187+00:00,food-allergy-data-dictionary,2022-07-19T14:57:42.614680+00:00,2,f881b5bc-5f27-434a-8ad9-a2d5f06964a9,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,FA research lacks a widely adopted terminologi...,The Food Allergy Data Dictionary,open,public,open,NaN
2,2022-07-19T14:57:42.428426+00:00,syncropatch-db-r-package,2022-07-19T14:57:42.501299+00:00,2,e780b050-d348-4e11-8489-0f464cac74c4,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,NaN,Syncropatch DB R Package,open,public,open,NaN
3,2022-07-19T14:57:42.177728+00:00,script-project,2022-07-19T14:57:42.229279+00:00,2,507f92f1-6152-47ec-b8d8-b065d05e71e6,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,project,Project,These are various artifacts from the SCRIPT pr...,SCRIPT Project,open,public,open,NaN
4,2022-07-19T14:57:42.068743+00:00,grupo-de-equilibrio-de-estilo-de-vida,2022-07-19T14:57:42.131976+00:00,2,84fc9255-0d2c-48ab-aaf2-83a75ae4a113,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/c...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,https://vtfsmghslrepo02.fsm.northwestern.edu/a...,topic,Topic,NaN,Grupo de Equilibrio De Estilo De Vida,open,public,open,NaN
